In [0]:
from pyspark.sql import functions as F

# Leer Silver
silver_df = spark.table("workspace.javier_altoe.propiedades_silver")

# ====== Transformaciones para Gold ======

# 1. Calcular precio por metro cuadrado
gold_df = silver_df.withColumn(
    "precio_m2", 
    F.when(F.col("metros_cuadrados_totales") > 0, F.col("precio") / F.col("metros_cuadrados_totales"))
     .otherwise(None)
)

# 2. Clasificación por antigüedad
gold_df = gold_df.withColumn(
    "categoria_antiguedad",
    F.when(F.col("antiguedad") < 5, "a_estrenar")
     .when((F.col("antiguedad") >= 5) & (F.col("antiguedad") < 20), "reciente")
     .when((F.col("antiguedad") >= 20) & (F.col("antiguedad") < 50), "media")
     .when(F.col("antiguedad") >= 50, "antiguo")
     .otherwise("desconocido")
)

# 3. Clasificación de rangos de precio
gold_df = gold_df.withColumn(
    "rango_precio",
    F.when(F.col("precio") < 50000, "bajo")
     .when((F.col("precio") >= 50000) & (F.col("precio") < 150000), "medio")
     .when((F.col("precio") >= 150000) & (F.col("precio") < 300000), "alto")
     .when(F.col("precio") >= 300000, "premium")
     .otherwise("desconocido")
)

# 4. Seleccionar solo las columnas definidas en el DDL de Gold
gold_df_final = gold_df.select(
    "id",
    "zona",
    "tipo_de_operacion",
    "precio",
    "precio_m2",
    "categoria_antiguedad",
    "rango_precio"
)

# 5. Guardar en la tabla Gold (detallada)
GOLD_TABLE = "workspace.javier_altoe.propiedades_gold"
gold_df_final.write.format("delta").mode("overwrite").saveAsTable(GOLD_TABLE)

print("✅ Datos enriquecidos y guardados en Gold:", GOLD_TABLE)


In [0]:
%sql
select *
from workspace.javier_altoe.propiedades_gold;